In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install -q keras

In [0]:
import os,sys,inspect
import os
import joblib
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import tensorflow as tf

from tensorflow.keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Input, Dense, Activation, Dropout, LeakyReLU, Softmax, ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
import process_data

In [0]:
# # Load dataset from local

# from google.colab import files
# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
 
!unzip -o data.zip

In [110]:
# Dataset loading

A, X, Y, train_idx, val_idx, test_idx = process_data.load_data("cora")
X = process_data.preprocess_features(X)

(2708, 2708)
(2708, 1433)


In [0]:
def mask_data(X, idx):
  new = np.zeros(X.shape)
  mask = np.zeros(X.shape[0]).astype(np.bool)
  mask[idx] = 1
  new[mask, :] = X[mask, :]
  
  return new

In [0]:
# Create attention

class Attention(Layer):
  
  def __init__(self, F_1, kernel_regularizer, **kwargs):
    self.F_1 = F_1
    self.kernel_regularizer = kernel_regularizer
    super(Attention, self).__init__(**kwargs)

  def build(self, input_shape):
    F = input_shape[0][-1]
    self.W = self.add_weight(name='W', 
                             shape=(self.F_1, F),
                             initializer='glorot_uniform',
                             regularizer=self.kernel_regularizer,
                             trainable=True)

    self.a = self.add_weight(name='a', 
                             shape=(2, self.F_1, 1),
                             initializer='glorot_uniform',
                             regularizer=self.kernel_regularizer,
                             trainable=True)

    super(Attention, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]

    x_features = X @ tf.transpose(self.W)
#     x_features = (N, F_1)
    att_self = x_features @ self.a[0]
    att_neighbours = x_features @ self.a[1]
#     both att = (N, 1)
    att = att_self + tf.transpose(att_neighbours)
#     att = (N, N)
    att = LeakyReLU(alpha=0.2)(att)

    mask = -10e9 * (1.0 - A)
    att_masked = att + mask
#     att_masked = att * A
    dense = Softmax(axis=0)(att_masked)
    
#     dense = (N, N)
    dense = dense @ x_features
#     dense = (N, F_1)

    return dense

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.output_dim)
    


# Create Gaussian (kernel) layer

class GAT(Layer):
  
  def __init__(self, F_1, K, kernel_regularizer, last=False, **kwargs):
    self.F_1 = F_1
    self.K = K
    self.kernel_regularizer = kernel_regularizer
    self.last = last
    
    self.attentions = []
    for k in range(self.K):
      attention = Attention(self.F_1, self.kernel_regularizer)
      self.attentions.append(attention)
      
    super(GAT, self).__init__(**kwargs)
  
  def build(self, input_shape):

    super(GAT, self).build(input_shape)  # Be sure to call this at the end
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]

    attentions = []
    for k in range(self.K):
      attention = self.attentions[k]([X, A])
      attentions.append(attention)

    if self.last:
#       h = (N, F_1)
      h = tf.reduce_mean(tf.stack(attentions), axis=0)
      h = Softmax()(h)
    else:
#       h = (N, F_1*k)
      h = tf.concat(attentions, axis=1)
      h = ELU()(h)
    return h

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.output_dim)

In [0]:
epochs=300
batch_size = X.shape[0]
n_classes = 7
F_1 = 8
K = 8
dropout = 0.5
learning_rate = 5e-3
reg = l2(5e-4)

In [0]:
X_in = Input(shape=(X.shape[-1],))
A_in = Input(shape=(batch_size,))

dropout1 = Dropout(dropout)(X_in)
layer1 = GAT(F_1=F_1,
             K=K,
             kernel_regularizer=reg,
             last=False,
             trainable=True,
             input_shape=(X.shape[-1],))([dropout1, A_in])
dropout2 = Dropout(dropout)(layer1)
layer2 = GAT(F_1=n_classes,
             K=1,
             kernel_regularizer=reg,
             last=True,
             trainable=True)([dropout2, A_in])

model = Model(inputs=[X_in, A_in], outputs=layer2)

model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [115]:
validation_data = ([mask_data(X, val_idx), A], mask_data(Y, val_idx))
model.fit([mask_data(X, train_idx), A], mask_data(Y, train_idx), validation_data=validation_data, epochs=epochs, batch_size=batch_size, shuffle=False)

Train on 2708 samples, validate on 2708 samples
Epoch 1/300
2708/2708 [==============================] - 2s 702us/sample - loss: 0.1791 - accuracy: 0.5266 - val_loss: 0.4247 - val_accuracy: 0.3143
Epoch 2/300
2708/2708 [==============================] - 0s 114us/sample - loss: 0.1660 - accuracy: 0.5218 - val_loss: 0.4130 - val_accuracy: 0.3781
Epoch 3/300
2708/2708 [==============================] - 0s 112us/sample - loss: 0.1543 - accuracy: 0.5565 - val_loss: 0.4031 - val_accuracy: 0.3431
Epoch 4/300
2708/2708 [==============================] - 0s 103us/sample - loss: 0.1443 - accuracy: 0.5975 - val_loss: 0.3949 - val_accuracy: 0.3892
Epoch 5/300
2708/2708 [==============================] - 0s 105us/sample - loss: 0.1361 - accuracy: 0.5447 - val_loss: 0.3883 - val_accuracy: 0.4202
Epoch 6/300
2708/2708 [==============================] - 0s 104us/sample - loss: 0.1294 - accuracy: 0.5971 - val_loss: 0.3830 - val_accuracy: 0.4442
Epoch 7/300
2708/2708 [==============================] - 0